<a href="https://colab.research.google.com/github/basakstuff/DeepConvLSTM/blob/main/DCLSTM_TASK_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Task B2: Multimodal activity recognition: Gestures**

Including Relevent Library files.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import time
import pickle as cp
from sliding_window import sliding_window

Loading Opportunity Dataset and including accessory files.

In [ ]:
#Loading Opportunity Dataset.
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip --no-check-certificate

In [ ]:
#Including preprocessing file.
!python preprocess_data.py -h

usage: preprocess_data.py [-h] -i INPUT -o OUTPUT [-t {gestures,locomotion}]

Preprocess OPPORTUNITY dataset

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        OPPORTUNITY zip file
  -o OUTPUT, --output OUTPUT
                        Processed data file
  -t {gestures,locomotion}, --task {gestures,locomotion}
                        Type of activities to be recognized


## **TASK B :- Applying Model on Gestures (with NULL Class)**

Preprocessing the data from data set.

In [ ]:
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data -t gestures

Checking dataset data/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... file OpportunityUCIDataset/dataset/S2-ADL4.dat
... file OpportunityUCIDataset/dataset/S2-ADL5.dat
... file OpportunityUCIDataset/dataset/S3-ADL4.dat
... file OpportunityUCIDataset/dataset/S3-ADL5.dat
Fi

In [ ]:
# Hardcoded number of sensor channels employed in the OPPORTUNITY challenge
NB_SENSOR_CHANNELS = 113

# Hardcoded number of classes in the gesture recognition problem
NUM_CLASSES = 18

# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 24


# Hardcoded step of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_STEP = 12

# Batch Size

BATCH_SIZE = 100

# Number filters convolutional layers
NUM_FILTERS = 64

# Size filters convolutional layers
FILTER_SIZE = 5

# Number of unit in the long short-term recurrent layers
NUM_UNITS_LSTM = 128

Loading the data and segmenting it according to length of sliding window.

In [ ]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))

Loading data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


In [ ]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))
X_train.shape

 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)


(46495, 24, 113, 1)

In [ ]:
X_train.shape

(46495, 24, 113, 1)

Defining DeepConvLSTM Model. 

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential()
model.add(keras.Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1)))
initializer = tf.keras.initializers.Orthogonal()
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Permute((2,1,3)))
model.add(layers.Reshape( (int(model.layers[4].output_shape[1]), int(model.layers[4].output_shape[2]) * int(model.layers[4].output_shape[3]))))
model.add(layers.LSTM(NUM_UNITS_LSTM,return_sequences=True))
model.add(layers.LSTM(NUM_UNITS_LSTM,return_sequences=True))
#model.add(layers.Reshape( (-1, NUM_UNITS_LSTM)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASSES, activation="softmax"))

    
#Printing Model Summary.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 113, 64)       384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 113, 64)       20544     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 113, 64)       20544     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 113, 64)        20544     
_________________________________________________________________
permute (Permute)            (None, 113, 8, 64)        0         
_________________________________________________________________
reshape (Reshape)            (None, 113, 512)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 113, 128)          3

In [ ]:
model.layers[4].output_shape

(None, 113, 8, 64)

Encoding the training and testing data to One-Hot Encoded form.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
def prepare_targets(y_train, y_test):
	ohe = OneHotEncoder()
	ohe.fit(y_train)
	y_train_enc = ohe.transform(y_train)
	y_test_enc = ohe.transform(y_test)
	return y_train_enc.A, y_test_enc.A

In [ ]:
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

Compiling Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=[tf.keras.metrics.CategoricalAccuracy()])

Training Model

In [ ]:
model.fit(X_train, y_train_enc, batch_size=BATCH_SIZE, epochs=50,validation_split=0.1)
model.save("B_Null_Basak.h5") #MODEL_NAME.h5

#Model is already trained and loaded directly for testing data.

Epoch 1/50
419/419 [==============================] - 65s 141ms/step - loss: 1.5771 - categorical_accuracy: 0.6681 - val_loss: 0.8032 - val_categorical_accuracy: 0.8144
Epoch 2/50
419/419 [==============================] - 57s 136ms/step - loss: 0.7115 - categorical_accuracy: 0.7749 - val_loss: 0.7638 - val_categorical_accuracy: 0.8024
Epoch 3/50
419/419 [==============================] - 57s 136ms/step - loss: 0.5099 - categorical_accuracy: 0.8311 - val_loss: 0.7819 - val_categorical_accuracy: 0.8108
Epoch 4/50
419/419 [==============================] - 57s 136ms/step - loss: 0.4100 - categorical_accuracy: 0.8656 - val_loss: 0.6395 - val_categorical_accuracy: 0.8406
Epoch 5/50
419/419 [==============================] - 57s 136ms/step - loss: 0.3279 - categorical_accuracy: 0.8894 - val_loss: 0.6909 - val_categorical_accuracy: 0.8303
Epoch 6/50
419/419 [==============================] - 57s 136ms/step - loss: 0.2749 - categorical_accuracy: 0.9079 - val_loss: 0.6959 - val_categorical_acc

In [ ]:
y_train.shape

(46495, 1)

In [ ]:
from tensorflow.keras.models import load_model

new_model3 = load_model('B_Null_Basak.h5')

Testing our Model on test data.

In [ ]:
y_prob = new_model3.predict(X_test) 
y_classes = y_prob.argmax(axis=-1)

In [ ]:
print(y_classes.shape)

(9894,)


Printing F1 Score and relevent Result Metrics.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1_score, supp = precision_recall_fscore_support(y_classes, y_test, average='weighted')

from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_classes, y_test))
#print(metrics)
print("PRECISION: {0}" .format(precision))
print("RECALL: {0}".format(recall))
print("F1_SCORE : {0}".format(f1_score))


[[7853   15   16    8    9  102   50   41   42   13   27   22    7   20
    16   63  155   46]
 [   7   31    0    7    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   7    0   57    0    2    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   1   11    0   45    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   9    0   22    0   72    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [  23    0    0    0    0   99    4    5    0    1    0    0    0    0
     0    0    0    0]
 [  24    0    0    0    0   13   94    0    3    2    0    0    1    0
     0    1    0    0]
 [  36    0    0    0    0   11    2   49    5    2    0    1    0    1
     0    0    6    0]
 [  23    0    0    0    0    3    8    0   23    0    0    0    0    1
     0    0    0    0]
 [   7    1    0    0    0    0    1    0    0   10    4    5    5    1
     0    0    0    3]
 [   7    0    0    0    0    0    1    0    0    

--------------------------------------------------------------------------------

## **TASK B: Applying Model on Gesture dataset (without NULL class)**

Preprocessing the data from data set.

In [ ]:
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data -t gestures

Checking dataset data/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... file OpportunityUCIDataset/dataset/S2-ADL4.dat
... file OpportunityUCIDataset/dataset/S2-ADL5.dat
... file OpportunityUCIDataset/dataset/S3-ADL4.dat
... file OpportunityUCIDataset/dataset/S3-ADL5.dat
Fi

Defining important variables.

In [ ]:
# Hardcoded number of sensor channels employed in the OPPORTUNITY challenge
NB_SENSOR_CHANNELS = 113

# Hardcoded number of classes in the gesture recognition problem
NUM_CLASSES = 17

# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 24


# Hardcoded step of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_STEP = 12

# Batch Size

BATCH_SIZE = 100

# Number filters convolutional layers
NUM_FILTERS = 64

# Size filters convolutional layers
FILTER_SIZE = 5

# Number of unit in the long short-term recurrent layers
NUM_UNITS_LSTM = 128

Loading Data an segmenting data according to sliding window length.

In [ ]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))

Loading data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


In [ ]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))
print(X_train.shape)
print(y_train.shape)

 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)
(46495, 24, 113, 1)
(46495,)


Defining DeepConvLSTM Model.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential()
model.add(keras.Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1)))

#intializing weights
initializer = tf.keras.initializers.Orthogonal()

#Adding 4 CNN layers.
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu", kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Permute((2,1,3)))
model.add(layers.Reshape( (int(model.layers[4].output_shape[1]), int(model.layers[4].output_shape[2]) * int(model.layers[4].output_shape[3]))))

#Adding 2 LSTM layers.
model.add(layers.LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model.add(layers.LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
#model.add(layers.Reshape( (-1, NUM_UNITS_LSTM)))
model.add(layers.Flatten())

#Applying a dense layer of softmax.
model.add(layers.Dense(NUM_CLASSES, activation="softmax"))


model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 20, 113, 64)       384       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 113, 64)       20544     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 113, 64)       20544     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 113, 64)        20544     
_________________________________________________________________
permute_1 (Permute)          (None, 113, 8, 64)        0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 113, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 113, 128)         

In [ ]:
model.layers[4].output_shape

(None, 113, 8, 64)

Encoding the training and testing data to One-Hot Encoded form.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
def prepare_targets(y_train, y_test):
	ohe = OneHotEncoder()
	ohe.fit(y_train)
	y_train_enc = ohe.transform(y_train)
	y_test_enc = ohe.transform(y_test)
	return y_train_enc.A, y_test_enc.A

Removing NULL class values.

In [ ]:
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

count = 0
idx = []
for i in range(0,y_train.shape[0]):
  if(y_train[i] == 0):
    count += 1
    idx.append(i)
print(len(idx))
y_train_new = np.delete(y_train,idx)
y_train_new = y_train_new.reshape(-1,1)
print(y_train_new.shape)

print(X_train.shape)
X_train_new = np.delete(X_train,idx,axis=0)
print(X_train_new.shape)

count = 0
idx = []
for i in range(0,y_test.shape[0]):
  if(y_test[i] == 0):
    count += 1
    idx.append(i)
print(len(idx))
y_test_new = np.delete(y_test,idx,axis = 0)
print(y_test_new.shape)

X_test_new = np.delete(X_test,idx,axis = 0)
print(X_test_new.shape)

y_train_enc, y_test_enc = prepare_targets(y_train_new, y_test_new)
print(y_train_enc.shape)
print(y_test_enc.shape)


32348
(14147, 1)
(46495, 24, 113, 1)
(14147, 24, 113, 1)
8237
(1657, 1)
(1657, 24, 113, 1)
(14147, 17)
(1657, 17)


Compiling Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=[tf.keras.metrics.CategoricalAccuracy()])

Training Model

In [ ]:
model.fit(X_train_new, y_train_enc, batch_size=BATCH_SIZE, epochs=50,validation_split=0.1)
model.save("B_NoNull_Basak.h5")

#Model is already trained and loaded for testing directly.

Epoch 1/50
128/128 [==============================] - 23s 149ms/step - loss: 2.3564 - categorical_accuracy: 0.2716 - val_loss: 2.2409 - val_categorical_accuracy: 0.3731
Epoch 2/50
128/128 [==============================] - 18s 137ms/step - loss: 1.3927 - categorical_accuracy: 0.4980 - val_loss: 1.8694 - val_categorical_accuracy: 0.4495
Epoch 3/50
128/128 [==============================] - 18s 137ms/step - loss: 1.0575 - categorical_accuracy: 0.6095 - val_loss: 2.1049 - val_categorical_accuracy: 0.4495
Epoch 4/50
128/128 [==============================] - 18s 137ms/step - loss: 0.8465 - categorical_accuracy: 0.6778 - val_loss: 1.8880 - val_categorical_accuracy: 0.4728
Epoch 5/50
128/128 [==============================] - 18s 138ms/step - loss: 0.7166 - categorical_accuracy: 0.7219 - val_loss: 2.0134 - val_categorical_accuracy: 0.5208
Epoch 6/50
128/128 [==============================] - 18s 138ms/step - loss: 0.6111 - categorical_accuracy: 0.7625 - val_loss: 2.0287 - val_categorical_acc

Loading Model.

In [ ]:
from tensorflow.keras.models import load_model

new_model4 = load_model('B_NoNull_Basak.h5') #MODEL_NAME.h5

Testing our Model on test data.

In [ ]:
y_prob = new_model4.predict(X_test_new) 
#print(y_prob)

y_classes = y_prob.argmax(axis=-1)
for i in range(0,y_classes.shape[0]):
  y_classes[i] += 1



Printing F1 Score and relevent Result Metrics.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1_score, supp = precision_recall_fscore_support(y_classes, y_test_new, average='micro')

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_classes, y_test_new))
#print(metrics)
print("PRECISION: {0}" .format(precision))
print("RECALL: {0}".format(recall))
print("F1_SCORE : {0}".format(f1_score))

[[ 33   0  13   0   0   0   0   0   0   0   0   0   0   0   0   0   2]
 [  0  77   0   2   3   0   0   1   0   0   0   0   0   0   0   0   7]
 [ 24   1  46   2   0   0   0   0   0   0   0   0   0   0   4   6   1]
 [  0  17   0  79   0   0   0   0   0   0   0   0   0   0   1   0   0]
 [  0   0   0   0 133  11  14   1   1   0   8   1   4   0  16   2   3]
 [  0   0   0   0  29 121   3   4   0   1   0   1   1   1   3   8   3]
 [  0   0   0   0  37  11  61  14   0   0   4   0   0   1   0  15   0]
 [  0   0   1   0   4   8   7  41   0   1   0   1   0   0   0   2   0]
 [  0   0   0   0  17   7   2   2  22   9   6   4   1   0   0  33  16]
 [  0   0   0   0   4   1   0   0   5  26   4   3   0   0   0   2   4]
 [  1   0   0   0   0   0   2   1   0   1   4   0   2   0   1   0   0]
 [  0   0   0   0   0   0   0   3   1   0   5   9   2   8   0   0   0]
 [  0   0   0   0   0   0   8   7   2   0   9   5  45  16   0   0   0]
 [  0   0   0   0   0   1   3   2   2   0   0   2  12  35   0   0   0]
 [  0 